In [42]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

RSEED = 42

In [ ]:
df_train_prepro = pd.read_csv('data/data_with_distance.csv')
df_train_prepro.dtypes

departure_point        object
arrival_point          object
departure_time         object
arrival_time           object
flight_status          object
aircraft_code          object
target                float64
duration              float64
dep_hour                int64
dep_day                 int64
dep_month               int64
dep_dayofweek           int64
dep_quarter             int64
dep_season              int64
dep_is_weekend          int64
dep_time_of_day        object
arr_hour                int64
arr_day                 int64
arr_month               int64
arr_dayofweek           int64
arr_quarter             int64
arr_season              int64
arr_is_weekend          int64
arr_time_of_day        object
route                  object
departure_date         object
arrival_date           object
dep_temp              float64
dep_precip            float64
dep_wind              float64
arr_temp              float64
arr_precip            float64
arr_wind              float64
Country   

In [3]:
y = df_train_prepro['target']
X = df_train_prepro.drop(columns=['target'])

In [4]:
# Define the categorical features
num_col = ['duration','dep_temp', 'dep_precip', 'dep_wind', 'arr_temp',
       'arr_precip', 'arr_wind', 'holiday_length', 'num_passenger_year', 'distance_km']
cat_col = ['departure_point', 'arrival_point', 'flight_status', 'aircraft_code','dep_hour',
       'dep_day', 'dep_month', 'dep_dayofweek', 'dep_quarter', 'dep_season',
       'dep_is_weekend', 'dep_time_of_day', 'arr_hour', 'arr_day', 'arr_month',
       'arr_dayofweek', 'arr_quarter', 'arr_season', 'arr_is_weekend',
       'arr_time_of_day', 'route', 'is_holiday', 'Country', 'City', 'aircraft_model']

In [5]:
# Use sparse output for OneHotEncoder to save memory
encoder = OneHotEncoder(handle_unknown='ignore')
scaler = StandardScaler()

# Fit and transform categorical columns (sparse matrix)
X_cat_sparse = encoder.fit_transform(X[cat_col])

# Scale only the numerical columns and convert to float32
X_num_scaled = scaler.fit_transform(X[num_col]).astype(np.float32)

# Convert sparse matrix to float32 and combine with numerical features
from scipy import sparse
X_encoded_scaled = sparse.hstack([X_num_scaled, X_cat_sparse.astype(np.float32)]).tocsr()

# Split the encoded and scaled data
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(
    X_encoded_scaled, y, stratify=y, test_size=0.2, random_state=RSEED
)

In [6]:
# Get indices for the split
train_idx, test_idx = train_test_split(
    np.arange(len(X)), stratify=y, test_size=0.2, random_state=RSEED
)

X_train_1_raw = X.iloc[train_idx]
X_train_2_raw = X.iloc[test_idx]
y_train_1_raw = y.iloc[train_idx]
y_train_2_raw = y.iloc[test_idx]

# Get categorical column indices for CatBoost
cat_features_idx = [X.columns.get_loc(col) for col in cat_col]

Create Base models + Meta model

In [ ]:
# Base models
random_forest = RandomForestRegressor(random_state=RSEED)
xgb = XGBRegressor(objective='reg:squarederror', random_state=RSEED)
ridge = Ridge(random_state=RSEED)
knn = KNeighborsRegressor()
lgbm = LGBMRegressor(random_state=RSEED)
catboost = CatBoostRegressor(verbose=0, random_state=RSEED)
adaboost = AdaBoostRegressor(n_estimators=100, random_state=RSEED)
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RSEED)
svr = SVR()

In [45]:
# Meta-model to combine all the base models
#linear_model = LinearRegression()
meta_xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

In [55]:
# Fit all the base estimators on the 1st half of the train dataset

#Fit Random Forest
rf_model = random_forest.fit(X_train_1, y_train_1) 

In [9]:
#Fit XGBoost
xgb_model = xgb.fit(X_train_1, y_train_1)

In [10]:
# Fit Ridge Convert sparse matrix to dense for Ridge regression
ridge_model = Ridge(random_state=42).fit(X_train_1.toarray(), y_train_1)

In [11]:
# fit KNN KNeighborsRegressor does not support sparse input, so convert to dense
knn_model = knn.fit(X_train_1, y_train_1)

In [ ]:
# Fit LightGBM
lgbm_model = lgbm.fit(X_train_1, y_train_1)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3680
[LightGBM] [Info] Number of data points in the train set: 79797, number of used features: 804
[LightGBM] [Info] Start training from score 9.881625


In [49]:
# Drop datetime columns that CatBoost cannot handle
datetime_cols = ['departure_time', 'arrival_time', 'departure_date', 'arrival_date']
X_train_1_raw_catboost = X_train_1_raw.drop(columns=datetime_cols)

# Update categorical feature indices for the new dataframe
cat_features_idx_catboost = [X_train_1_raw_catboost.columns.get_loc(col) for col in cat_col if col in X_train_1_raw_catboost.columns]

catboost_model = catboost.fit(X_train_1_raw_catboost, y_train_1_raw, cat_features=cat_features_idx_catboost)

In [14]:
# Fit AdaBoost on encoded/scaled training data
adaboost_model = adaboost.fit(X_train_1, y_train_1)

In [15]:
# Fit GradientBoostingRegressor on encoded/scaled training data
gbr_model = gbr.fit(X_train_1, y_train_1)

In [ ]:
# Fit support vector regression - SVR does not support sparse input, so convert to dense
svr_model = svr.fit(X_train_1.toarray(), y_train_1)

In [56]:
# Predict the values from the base estimators with the second half of the train dataset
rf_pred = rf_model.predict(X_train_2)
xgb_pred = xgb_model.predict(X_train_2)
ridge_pred = ridge_model.predict(X_train_2)
knn_pred = knn_model.predict(X_train_2)
lgbm_pred = lgbm_model.predict(X_train_2)
catboost_pred = catboost_model.predict(X_train_2_raw.drop(columns=datetime_cols))
adaboost_pred = adaboost_model.predict(X_train_2)
gbr_pred = gbr_model.predict(X_train_2)
svr_pred = svr_model.predict(X_train_2.toarray())

# Combine base model predictions for meta-model input
combine_X_pred_test = pd.concat([
	pd.DataFrame(rf_pred),
	pd.DataFrame(xgb_pred),
	pd.DataFrame(ridge_pred),
    pd.DataFrame(knn_pred),
    pd.DataFrame(lgbm_pred),
    pd.DataFrame(catboost_pred),
    pd.DataFrame(adaboost_pred),
    pd.DataFrame(gbr_pred),
    pd.DataFrame(svr_pred)
], axis=1)

In [57]:
# Fit the final estimator on the combined probabilities and target values

# Ensure unique column names for stacking features
combine_X_pred_test.columns = [f'base_model_{i}' for i in range(combine_X_pred_test.shape[1])]

#linear_model.fit(combine_X_pred_test, y_train_2)
meta_xgb.fit(combine_X_pred_test, y_train_2)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [58]:
# Predict with meta-model
#y_pred = linear_model.predict(combine_X_pred_test)
y_pred = meta_xgb.predict(combine_X_pred_test)

In [59]:
# Ensure predictions are non-negative
y_pred[y_pred < 0] = 0
y_train_2 = y_train_2.clip(lower=0)

In [60]:
# Evaluate the model
mse = mean_squared_error(y_train_2, y_pred)
r2 = r2_score(y_train_2, y_pred)
rmse = np.sqrt(mean_squared_error(y_train_2, y_pred))
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')
print(f"Stacking RMSE: {rmse:.2f}")

Mean Squared Error: 442.10936673682323
R2 Score: 0.6694729274088058
Stacking RMSE: 21.03


In [67]:
rf_pred[rf_pred < 0] = 0
xgb_pred[xgb_pred < 0] = 0
ridge_pred[ridge_pred < 0] = 0
knn_pred[knn_pred < 0] = 0
lgbm_pred[lgbm_pred < 0] = 0
catboost_pred[catboost_pred < 0] = 0
adaboost_pred[adaboost_pred < 0] = 0
gbr_pred[gbr_pred < 0] = 0
svr_pred[svr_pred < 0] = 0
print(f"RF RMSE: {np.sqrt(mean_squared_error(y_train_2, rf_pred))}")
print(f"XGB RMSE: {np.sqrt(mean_squared_error(y_train_2, xgb_pred))}")
print(f"Ridge RMSE: {np.sqrt(mean_squared_error(y_train_2, ridge_pred))}")
print(f"KNN RMSE: {np.sqrt(mean_squared_error(y_train_2, knn_pred))}")
print(f"LGBM RMSE: {np.sqrt(mean_squared_error(y_train_2, lgbm_pred))}")
print(f"CatBoost RMSE: {np.sqrt(mean_squared_error(y_train_2, catboost_pred))}")
print(f"AdaBoost RMSE: {np.sqrt(mean_squared_error(y_train_2, adaboost_pred))}")
print(f"GBR RMSE: {np.sqrt(mean_squared_error(y_train_2, gbr_pred))}")
print(f"SVR RMSE: {np.sqrt(mean_squared_error(y_train_2, svr_pred))}")


RF RMSE: 32.56811761138045
XGB RMSE: 33.054362511146095
Ridge RMSE: 33.78760614440644
KNN RMSE: 35.12305457112201
LGBM RMSE: 33.02512061324591
CatBoost RMSE: 31.019297727488464
AdaBoost RMSE: 35.97175375380947
GBR RMSE: 34.140939340766465
SVR RMSE: 34.89037931383889


In [62]:
# Get feature importances from each base model and display them

# Random Forest feature importances
rf_importances = rf_model.feature_importances_

# XGBoost feature importances
xgb_importances = xgb_model.feature_importances_

# KNN does not provide feature importances, so skip it

# Ridge feature coefficients (absolute value for importance)
ridge_importances = np.abs(ridge_model.coef_)

# LightGBM feature importances
lgbm_importances = lgbm_model.feature_importances_

# CatBoost feature importances
catboost_importances = catboost.get_feature_importance()

# AdaBoost feature importances (using the base estimator's feature importances)
adaboost_importances = adaboost_model.feature_importances_

# Gradient Boosting feature importances
gbr_importances = gbr_model.feature_importances_

# Get encoded categorical column names
encoded_cat_cols = encoder.get_feature_names_out(cat_col)

# Feature names
feature_names = num_col + list(encoded_cat_cols)

# Create a DataFrame for each model's importances
importances_df = pd.DataFrame({
    'feature': feature_names,
    'RandomForest': rf_importances,
    'XGBoost': xgb_importances,
    'Ridge': ridge_importances,
    'LightGBM': lgbm_importances,
    #'CatBoost': catboost_importances,
    'AdaBoost': adaboost_importances,
    'GradientBoosting': gbr_importances
})

# Show top 15 features by average importance across models
importances_df['avg_importance'] = importances_df[[
    'RandomForest',
    'XGBoost', 
    'Ridge', 
    'LightGBM',
    #'CatBoost',
    'AdaBoost',
    'GradientBoosting'
    ]].mean(axis=1)
importances_df.sort_values('avg_importance', ascending=False)

,feature,RandomForest,XGBoost,Ridge,LightGBM,AdaBoost,GradientBoosting,avg_importance
8,num_passenger_year,0.042434,0.003453,39.687557,218,0.043369,0.067736,42.974092
1,dep_temp,0.053690,0.000446,0.539253,153,0.000109,0.004199,25.599616
4,arr_temp,0.055669,0.000696,0.408382,135,0.000131,0.002531,22.577901
6,arr_wind,0.058381,0.000558,1.247772,121,0.000806,0.002400,20.384986
3,dep_wind,0.055698,0.000485,1.138566,114,0.002644,0.001387,19.199797
...,...,...,...,...,...,...,...,...
1093,route_TUN → AAE,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000
1069,route_TLS → NCE,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000
1059,route_TBJ → JED,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000
1041,route_SKX → TUN,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000


Further work - HYPERPARAMETERS - XGB RandomisedSearchCV

Find the best parameters for XGBRegressor

In [ ]:
# Fit model to training data
xgb_best = XGBRegressor(objective='reg:squarederror', random_state=42)

#hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1, 0.2, 0.5],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight': [1, 2]
}

xgb_search = GridSearchCV(
    XGBRegressor(objective='reg:squarederror', random_state=42), 
    xgb_param_grid, 
    cv=3, 
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

xgb_ransearcv_best = RandomizedSearchCV(estimator=xgb_best, param_distributions=xgb_param_grid, 
                          n_iter=100, scoring='accuracy', cv=3, 
                          verbose=1, random_state=42, n_jobs=-1)
xgb_ransearcv_best.fit(X_train_1, y_train_1)

In [ ]:
from sklearn.model_selection import GridSearchCV

rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0]
}

lgbm_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Random Forest
rf_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
rf_search.fit(X_train_1, y_train_1)
best_rf = rf_search.best_estimator_

# XGBoost
xgb_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

# Ridge
ridge_search = GridSearchCV(Ridge(random_state=42), ridge_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
ridge_search.fit(X_train_1, y_train_1)
best_ridge = ridge_search.best_estimator_

# LightGBM
lgbm_search = GridSearchCV(LGBMRegressor(random_state=42), lgbm_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
lgbm_search.fit(X_train_1, y_train_1)
best_lgbm = lgbm_search.best_estimator_